In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torch
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fixed columns
cols_standardize = ['Age','BMI', 'poverty_level', 'PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9']
cols_leave = ['Race', 'sex', 'Mobility', 'diabetes.y', 'Asthma', 'Arthritis', 'heart_failure', 'coronary_heart_disease', 
              'angina', 'stroke', 'thyroid', 'bronchitis', 'cancer']

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]
x_mapper = DataFrameMapper(standardize + leave)

# Model architecture
def get_model(input_dim):
    num_nodes = [256, 256]
    net = tt.practical.MLPVanilla(input_dim, num_nodes, 1, batch_norm=True, dropout=0.7, output_bias=False).to(device)
    return CoxPH(net, tt.optim.Adam)

# For C-index storage
c_index_list = []

# Directory containing the splits
split_dir = "D:/Final_Year_Project/splits_final/"  # Modify this if needed

for i in range(1, 101):
    print(f"Processing split {i}...")

    # Load train/test split
    df_train = pd.read_csv(os.path.join(split_dir, f"train_split_{i}.csv"))
    df_test = pd.read_csv(os.path.join(split_dir, f"test_split_{i}.csv"))

    # Fit transformer only on training data
    x_train = torch.tensor(x_mapper.fit_transform(df_train).astype('float32')).to(device)
    x_val = torch.tensor(x_mapper.transform(df_test).astype('float32')).to(device)
    x_test = torch.tensor(x_mapper.transform(df_test).astype('float32')).to(device)

    # Get target
    get_target = lambda df: (
        torch.tensor(df['time_mort'].values, dtype=torch.float32).to(device),
        torch.tensor(df['mortstat'].values, dtype=torch.float32).to(device)
    )
    y_train = get_target(df_train)
    y_val = get_target(df_test)
    durations_test, events_test = get_target(df_test)
    val = x_val, y_val

    durations_np = durations_test.cpu().numpy()
    events_np = events_test.cpu().numpy()

    # Model
    model = get_model(x_train.shape[1])

    # Find LR
    batch_size = 64
    lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()
    model.optimizer.set_lr(best_lr)

    # Train
    callbacks = [tt.callbacks.EarlyStopping()]
    model.fit(x_train, y_train, batch_size, 512, callbacks, verbose=False, 
              val_data=val, val_batch_size=batch_size)

    # Predict survival
    model.compute_baseline_hazards()
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_np, events_np, censor_surv='km')
    c_index = ev.concordance_td()
    c_index_list.append(c_index)

# Save C-indices
c_index_df = pd.DataFrame({'split': list(range(1, 101)), 'c_index': c_index_list})
# c_index_df.to_csv("D:/Final_Year_Project/c_index_results.csv", index=False)

print("All splits processed. Results saved to c_index_results.csv.")

Processing split 1...


C:\Users\gh22s\AppData\Local\Programs\Python\Python310\lib\site-packages\torchtuples\base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torc

Processing split 2...


C:\Users\gh22s\AppData\Local\Programs\Python\Python310\lib\site-packages\torchtuples\base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torc

Processing split 3...


...

Processing split 99...


C:\Users\gh22s\AppData\Local\Programs\Python\Python310\lib\site-packages\torchtuples\base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torc

Processing split 100...


C:\Users\gh22s\AppData\Local\Programs\Python\Python310\lib\site-packages\torchtuples\base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torc

All splits processed. Results saved to c_index_results.csv.


In [2]:
c_index_df['c_index'].mean()

np.float64(0.7812455008664488)

In [3]:
c_index_df.to_csv("D:/Final_Year_Project/c_index_FPCdeepsurv.csv", index=False)